# SnappBox Optimization Demo Notebook

This notebook provides a live demonstration of the Smart Logistics & Delivery Optimization system, simulating a SnappBox-like scenario. It showcases how real-time data, routing algorithms, and RL-driven fleet management interact to optimize deliveries.

In [ ]:
import requests
import json
import time
import datetime
import pandas as pd
import numpy as np
import os
import networkx as nx
import osmnx as ox
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, HTML
import warnings

warnings.filterwarnings('ignore')

## 1. Configuration and API Endpoints

In [ ]:
API_BASE_URL = "http://localhost:8000"
ROUTE_ENDPOINT = f"{API_BASE_URL}/route"
OPTIMIZE_FLEET_ENDPOINT = f"{API_BASE_URL}/optimize_fleet"
UPDATE_TRAFFIC_ENDPOINT = f"{API_BASE_URL}/update_traffic"

SCENARIO_PATH = 'data_nexus/simulation_scenarios/tehran_fleet_scenario.pkl'

try:
    with open(SCENARIO_PATH, 'rb') as f:
        scenario = json.load(f) # Assuming scenario is stored as JSON for easier loading
except (FileNotFoundError, json.JSONDecodeError):
    print(f"Warning: Scenario file not found or invalid at {SCENARIO_PATH}. Generating dummy data.")
    # Generate dummy data for demo if scenario file is missing
    from data_nexus.simulation_scenarios.generate_tehran_fleet_env import TehranFleetEnvironmentGenerator
    osm_config_path = 'conf/osm_processing_config.yaml'
    if not os.path.exists('data_nexus/road_network_graph/preprocessed_tehran_graph.gml'):
        print("Preprocessed graph not found. Running OSMnx processor.")
        from src.graph_routing_engine.osmnx_processor import OSMNxProcessor
        processor = OSMNxProcessor(osm_config_path)
        processor.download_and_process_graph()
    
    generator = TehranFleetEnvironmentGenerator(osm_config_path)
    drivers_df = generator.generate_drivers(5)
    orders_df = generator.generate_orders(10)
    traffic_df = generator.generate_traffic_data(20)
    
    # Convert to dicts for JSON serialization and then back for scenario structure
    scenario = {
        'drivers': drivers_df.to_dict('records'),
        'orders': orders_df.to_dict('records'),
        'traffic': traffic_df.to_dict('records') # Only include serializable parts
    }
    # Save the dummy scenario as JSON for consistent loading
    os.makedirs(os.path.dirname(SCENARIO_PATH), exist_ok=True)
    with open(SCENARIO_PATH, 'w') as f:
        json.dump(scenario, f, default=str) # default=str handles datetime objects
    print("Dummy scenario generated and saved.")
    
drivers_data = scenario['drivers']
orders_data = scenario['orders']
traffic_events = scenario['traffic']

print("API endpoints loaded.")

## 2. Helper Functions for Simulation

In [ ]:
def call_route_api(origin_lat, origin_lon, dest_lat, dest_lon, vehicle_profile='car'):
    request_payload = {
        "origin": {"lat": origin_lat, "lon": origin_lon},
        "destination": {"lat": dest_lat, "lon": dest_lon},
        "vehicle_profile": vehicle_profile,
        "current_time_utc": datetime.datetime.utcnow().isoformat() + "Z"
    }
    try:
        response = requests.post(ROUTE_ENDPOINT, json=request_payload, timeout=5)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Routing API error: {e}")
        return None

def call_optimize_fleet_api(current_drivers, pending_orders):
    driver_statuses = []
    for driver in current_drivers:
        driver_statuses.append({
            "driver_id": driver['driver_id'],
            "current_node_id": driver['current_node_id'],
            "vehicle_type": driver['vehicle_type'],
            "current_load": driver['current_load'],
            "capacity": driver['capacity'],
            "status": driver['status']
        })
    
    order_requests = []
    for order in pending_orders:
        order_requests.append({
            "order_id": order['order_id'],
            "pickup_node_id": order['origin_node'],
            "delivery_node_id": order['destination_node'],
            "weight": order['weight'],
            "volume": order['volume'],
            "pickup_time_start": order['pickup_time_start'],
            "pickup_time_end": order['pickup_time_end'],
            "delivery_time_latest": order['delivery_time_latest']
        })

    request_payload = {
        "current_drivers": driver_statuses,
        "pending_orders": order_requests,
        "current_time_utc": datetime.datetime.utcnow().isoformat() + "Z"
    }
    try:
        response = requests.post(OPTIMIZE_FLEET_ENDPOINT, json=request_payload, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Optimize Fleet API error: {e}")
        return None

def call_update_traffic_api(edge_u, edge_v, edge_key, travel_time):
    request_payload = {
        "edge_u": edge_u,
        "edge_v": edge_v,
        "edge_key": edge_key,
        "current_travel_time": travel_time,
        "timestamp_utc": datetime.datetime.utcnow().isoformat() + "Z"
    }
    try:
        response = requests.post(UPDATE_TRAFFIC_ENDPOINT, json=request_payload, timeout=2)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Update Traffic API error: {e}")
        return None


## 3. Real-time Simulation Loop

In [ ]:
# Load the preprocessed graph for visualization (not part of API calls)
osm_processing_config_path = 'conf/osm_processing_config.yaml'
with open(osm_processing_config_path, 'r') as f:
    osm_config = yaml.safe_load(f)
graph_path = osm_config['graph_serialization']['output_path']
main_graph = nx.read_gml(graph_path)

# Initialize simulation state
current_drivers_state = {d['driver_id']: d for d in drivers_data}
current_orders_state = {o['order_id']: o for o in orders_data}

simulation_metrics = []
simulation_time_step = 60 # seconds
total_simulation_duration = 3600 # seconds (1 hour)
current_sim_time = 0

print("Starting real-time simulation...")
while current_sim_time < total_simulation_duration:
    print(f"\n--- Simulation Time: {current_sim_time // 60:.0f} min {current_sim_time % 60:.0f} sec ---")

    # 1. Update Traffic (simulated)
    for event in traffic_events:
        event_timestamp = pd.to_datetime(event['timestamp'])
        if (event_timestamp - datetime.datetime.now()).total_seconds() < current_sim_time + simulation_time_step:
            call_update_traffic_api(event['u'], event['v'], event['key'], event['current_travel_time'])

    # 2. Call Fleet Optimization API
    pending_orders_list = [o for o in current_orders_state.values() if o['status'] == 'pending']
    drivers_list = list(current_drivers_state.values())

    optimization_response = call_optimize_fleet_api(drivers_list, pending_orders_list)

    if optimization_response:
        # Apply assignments
        for assignment in optimization_response['assignments']:
            driver_id = assignment['driver_id']
            for order_id in assignment['order_ids']:
                if order_id in current_orders_state:
                    current_orders_state[order_id]['status'] = 'assigned'
                    current_drivers_state[driver_id]['assigned_orders'] = current_drivers_state[driver_id].get('assigned_orders', []) + [order_id]
                    current_drivers_state[driver_id]['status'] = 'on_route'
                    print(f"  API: Assigned order {order_id} to driver {driver_id}")
            if 'target_node' in assignment:
                current_drivers_state[driver_id]['target_node'] = assignment['target_node']

        # Apply re-routes (simplified: just update driver's target node or path)
        for re_route in optimization_response['re_routes']:
            driver_id = re_route['driver_id']
            # Assuming 'new_route_points' implies a new path to follow
            if 'new_route_points' in re_route and re_route['new_route_points']:
                # For visualization, we can store these points
                current_drivers_state[driver_id]['current_route_path'] = re_route['new_route_points']
                print(f"  API: Driver {driver_id} re-routed.")

    # 3. Simulate Driver Movement and Order Completion
    for driver_id, driver in current_drivers_state.items():
        if driver['status'] == 'on_route' and driver.get('target_node') is not None:
            # Get current lat/lon from driver's current_node_id
            current_node = driver['current_node_id']
            target_node = driver['target_node']

            current_lat = main_graph.nodes[current_node]['y']
            current_lon = main_graph.nodes[current_node]['x']
            target_lat = main_graph.nodes[target_node]['y']
            target_lon = main_graph.nodes[target_node]['x']

            route_info = call_route_api(current_lat, current_lon, target_lat, target_lon)
            if route_info:
                # Simulate moving along a small segment of the route
                travel_distance_in_step = min(route_info['distance_meters'], driver.get('speed_mps', 10) * simulation_time_step)
                
                # A more sophisticated simulation would advance along the path geometry.
                # For this demo, just teleport to target if very close or update position notionally
                if route_info['distance_meters'] < 50: # Close enough to consider arrived
                    driver['current_node_id'] = target_node
                    driver['status'] = 'available'
                    driver['target_node'] = None
                    print(f"  Sim: Driver {driver_id} arrived at node {target_node}.")
                    
                    # Check for order pickups/deliveries at this node
                    orders_at_node = [o for oid, o in current_orders_state.items() if (o['origin_node'] == target_node and o['status'] == 'assigned') or (o['destination_node'] == target_node and o['status'] == 'picked_up')]
                    for order in orders_at_node:
                        if order['origin_node'] == target_node and order['status'] == 'assigned':
                            order['status'] = 'picked_up'
                            driver['current_load'] += order['weight']
                            print(f"  Sim: Driver {driver_id} picked up order {order['order_id']}.")
                        elif order['destination_node'] == target_node and order['status'] == 'picked_up':
                            order['status'] = 'delivered'
                            driver['current_load'] -= order['weight']
                            print(f"  Sim: Driver {driver_id} delivered order {order['order_id']}.")
                            if order['order_id'] in driver['assigned_orders']:
                                driver['assigned_orders'].remove(order['order_id'])

    # 4. Collect Metrics
    delivered_count = sum(1 for o in current_orders_state.values() if o['status'] == 'delivered')
    pending_count = sum(1 for o in current_orders_state.values() if o['status'] == 'pending' or o['status'] == 'assigned')
    in_transit_count = sum(1 for o in current_orders_state.values() if o['status'] == 'picked_up')

    simulation_metrics.append({
        'time': current_sim_time,
        'delivered': delivered_count,
        'pending': pending_count,
        'in_transit': in_transit_count,
        'available_drivers': sum(1 for d in current_drivers_state.values() if d['status'] == 'available')
    })

    current_sim_time += simulation_time_step
    time.sleep(1) # Pause for human readability

metrics_df = pd.DataFrame(simulation_metrics)
print("\nSimulation finished.")

## 4. Visualization of Results

In [ ]:
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
px.line(metrics_df, x='time', y=['delivered', 'pending', 'in_transit'], title='Order Status Over Time').show()

plt.subplot(1, 2, 2)
px.line(metrics_df, x='time', y='available_drivers', title='Available Drivers Over Time').show()

# Optional: Visualize final state of orders and drivers on a map
# Extract node coordinates from the main_graph
node_coords = pd.DataFrame.from_dict({
    node: {'y': data['y'], 'x': data['x']} for node, data in main_graph.nodes(data=True)
}, orient='index')
node_coords.index.name = 'node_id'

# Prepare data for plotting drivers
driver_locations = []
for drv_id, drv_state in current_drivers_state.items():
    if drv_state['current_node_id'] in node_coords.index:
        loc = node_coords.loc[drv_state['current_node_id']]
        driver_locations.append({
            'lat': loc['y'],
            'lon': loc['x'],
            'id': drv_id,
            'status': drv_state['status']
        })
drivers_df_plot = pd.DataFrame(driver_locations)

# Prepare data for plotting orders
order_locations = []
for ord_id, ord_state in current_orders_state.items():
    if ord_state['status'] != 'delivered': # Only show active orders
        pickup_loc = node_coords.loc[ord_state['origin_node']]
        delivery_loc = node_coords.loc[ord_state['destination_node']]
        order_locations.append({'lat': pickup_loc['y'], 'lon': pickup_loc['x'], 'type': 'Pickup', 'status': ord_state['status'], 'id': ord_id})
        order_locations.append({'lat': delivery_loc['y'], 'lon': delivery_loc['x'], 'type': 'Delivery', 'status': ord_state['status'], 'id': ord_id})
orders_df_plot = pd.DataFrame(order_locations)


fig = go.Figure()

# Plot drivers
if not drivers_df_plot.empty:
    fig.add_trace(go.Scattermapbox(
        lat=drivers_df_plot['lat'],
        lon=drivers_df_plot['lon'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=12,
            color=[('green' if s == 'available' else 'orange') for s in drivers_df_plot['status']],
            opacity=0.8
        ),
        text=drivers_df_plot.apply(lambda row: f"Driver {row['id']}<br>Status: {row['status']}", axis=1),
        name='Drivers'
    ))

# Plot orders
if not orders_df_plot.empty:
    fig.add_trace(go.Scattermapbox(
        lat=orders_df_plot['lat'],
        lon=orders_df_plot['lon'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=8,
            color=[('blue' if s == 'assigned' else 'purple') for s in orders_df_plot['status']],
            opacity=0.7
        ),
        text=orders_df_plot.apply(lambda row: f"Order {row['id']}<br>Type: {row['type']}<br>Status: {row['status']}", axis=1),
        name='Active Orders'
    ))

fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_center_lat=drivers_df_plot['lat'].mean() if not drivers_df_plot.empty else node_coords['y'].mean(),
    mapbox_center_lon=drivers_df_plot['lon'].mean() if not drivers_df_plot.empty else node_coords['x'].mean(),
    mapbox_zoom=10,
    title_text="Live Logistics Demo Map",
    margin={"r":0,"t":50,"l":0,"b":0}
)
fig.show()